# ResNet34 Analog Training with Tiki Taka Optimizer Example
    Training the ResNet34 neural network with Tiki Taka analog optimizer on CIFAR10 dataset, simulated on the analog resistive random-access memeory with soft bounds (ReRam) device.

In [ ]:
#%pip install aihwkit

In [1]:
def create_rpu_config():

    from aihwkit.simulator.presets import TikiTakaReRamSBPreset
    from aihwkit.simulator.configs.utils import MappingParameter

    # Define device and chip configuration used in the RPU tile
    mapping = MappingParameter(max_input_size=512,  # analog tile size
                            max_output_size=512,
                            digital_bias=True,
                            weight_scaling_omega=0.6)  # whether to use analog or digital bias

    rpu_config = TikiTakaReRamSBPreset(mapping=mapping)

    return rpu_config

In [ ]:
# Imports from PyTorch.
from torchvision.models import resnet34

# Imports from aihwkit.
from aihwkit.nn.conversion import convert_to_analog_mapped


def create_analog_netwrok(rpu_config):
    # Example: Load a predefined model from pytorch library and convert to
    #          its analog version.

    # Load a pytorch model.
    model = resnet34()
    model = convert_to_analog_mapped(model, rpu_config)

    return model


In [ ]:
from torch.nn import CrossEntropyLoss

criterion = CrossEntropyLoss()

from aihwkit.optim import AnalogSGD

def create_analog_optimizer(model):
    """Create the analog-aware optimizer.

    Args:
        model (nn.Module): model to be trained

    Returns:
        Optimizer: created analog optimizer
    """

    optimizer = AnalogSGD(model.parameters(), lr=0.1)
    optimizer.regroup_param_groups(model)

    return optimizer

In [ ]:
from torch import device, cuda

DEVICE = device('cuda' if cuda.is_available() else 'cpu')
print('Running the simulation on: ', DEVICE)

def train_step(train_data, model, criterion, optimizer):
    """Train network.

    Args:
        train_data (DataLoader): Validation set to perform the evaluation
        model (nn.Module): Trained model to be evaluated
        criterion (nn.CrossEntropyLoss): criterion to compute loss
        optimizer (Optimizer): analog model optimizer

    Returns:
        train_dataset_loss: epoch loss of the train dataset
    """
    total_loss = 0

    model.train()

    for images, labels in train_data:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()

        # Add training Tensor to the model (input).
        output = model(images)
        loss = criterion(output, labels)

        # Run training (backward propagation).
        loss.backward()

        # Optimize weights.
        optimizer.step()
        total_loss += loss.item() * images.size(0)
    train_dataset_loss = total_loss / len(train_data.dataset)

    return train_dataset_loss

In [ ]:
def test_step(validation_data, model, criterion):
    """Test trained network

    Args:
        validation_data (DataLoader): Validation set to perform the evaluation
        model (nn.Module): Trained model to be evaluated
        criterion (nn.CrossEntropyLoss): criterion to compute loss

    Returns: 
        test_dataset_loss: epoch loss of the train_dataset
        test_dataset_error: error of the test dataset
        test_dataset_accuracy: accuracy of the test dataset
    """
    total_loss = 0
    predicted_ok = 0
    total_images = 0

    model.eval()

    for images, labels in validation_data:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        pred = model(images)
        loss = criterion(pred, labels)
        total_loss += loss.item() * images.size(0)

        _, predicted = torch.max(pred.data, 1)
        total_images += labels.size(0)
        predicted_ok += (predicted == labels).sum().item()
        test_dataset_accuracy = predicted_ok/total_images*100
        test_dataset_error = (1-predicted_ok/total_images)*100

    test_dataset_loss = total_loss / len(validation_data.dataset)

    return test_dataset_loss, test_dataset_error, test_dataset_accuracy

In [ ]:
def training_loop(model, criterion, optimizer, train_data, validation_data, epochs=15, print_every=1):
    ###epochs
    """Training loop.

    Args:
        model (nn.Module): Trained model to be evaluated
        criterion (nn.CrossEntropyLoss): criterion to compute loss
        optimizer (Optimizer): analog model optimizer
        train_data (DataLoader): Validation set to perform the evaluation
        validation_data (DataLoader): Validation set to perform the evaluation
        epochs (int): global parameter to define epochs number
        print_every (int): defines how many times to print training progress

    """
    train_losses = []
    valid_losses = []
    test_error = []

    # Train model
    for epoch in range(0, epochs):
        # Train_step
        train_loss = train_step(train_data, model, criterion, optimizer)
        train_losses.append(train_loss)

        if epoch % print_every == (print_every - 1):
            # Validate_step
            with torch.no_grad():
                valid_loss, error, accuracy = test_step(validation_data, model, criterion)
                valid_losses.append(valid_loss)
                test_error.append(error)

            print(f'Epoch: {epoch}\t'
                  f'Train loss: {train_loss:.4f}\t'
                  f'Valid loss: {valid_loss:.4f}\t'
                  f'Test error: {error:.2f}%\t'
                  f'Test accuracy: {accuracy:.2f}%\t')

In [ ]:
import os
from torch import nn, Tensor
from torch.utils.data import DataLoader

from torchvision import datasets, transforms

# Path to store datasets
PATH_DATASET = os.path.join('data', 'DATASET')
os.makedirs(PATH_DATASET, exist_ok=True)

# Path to store results
RESULTS = os.path.join(os.getcwd(), 'results', 'RESNET')
WEIGHT_PATH = os.path.join(RESULTS, 'example_18_model_weight.pth')

# Training parameters
SEED = 1
N_EPOCHS = 60
BATCH_SIZE = 128
LEARNING_RATE = 0.1
N_CLASSES = 10


def load_images():
    """Load images for train from the torchvision datasets.

        Returns:
            Dataset, Dataset: train data and validation data    
    """
    mean = Tensor([0.4914, 0.4822, 0.4465])
    std = Tensor([0.2470, 0.2435, 0.2616])
    transform = datasets.Compose(
        [transforms.ToTensor(), transforms.Normalize(mean, std)])
    train_set = datasets.CIFAR10(PATH_DATASET, download=True, train=True, transform=transform)
    val_set = datasets.CIFAR10(PATH_DATASET, download=True, train=False, transform=transform)
    train_data = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
    validation_data = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

    return train_data, validation_data


In [ ]:
import torch

torch.manual_seed(1)

#load the dataset
train_data, test_data = load_images()

#create the rpu_config
rpu_config = create_rpu_config()

#create the model
model = create_analog_network(rpu_config).to(DEVICE)

#define the analog optimizer
optimizer = create_analog_optimizer(model)

training_loop(model, criterion, optimizer, train_data, test_data)